In [1]:
import pandas as pd
import os

In [2]:
# Load beneficiary, outpatient, and carrier claims files
ben_s1 = pd.read_csv("data\DE1_0_2008_Beneficiary_Summary_File_Sample_1.csv", low_memory = False)
op_s1 = pd.read_csv("data\DE1_0_2008_to_2010_Outpatient_Claims_Sample_1.csv", low_memory = False)
car_a_s1 = pd.read_csv("data\DE1_0_2008_to_2010_Carrier_Claims_Sample_1A.csv", low_memory = False)
#car_b_s1 = pd.read_csv("data\DE1_0_2008_to_2010_Outpatient_Claims_Sample_1.csv", low_memory = False)

In [4]:
## Create dataframe with state beneficiaries of interest
ben_s1_state = ben_s1[(ben_s1.SP_STATE_CODE == 31)]

len(ben_s1_state.index) #3,176 beneficiaries in 2008

3176

In [5]:
## Limit outpatient & carrier claims to just those for beneficiaries within state of interest
ben_s1_state_op = pd.merge(ben_s1_state,op_s1,on='DESYNPUF_ID',how='left')

ben_s1_state_cara = pd.merge(ben_s1_state,car_a_s1,on='DESYNPUF_ID',how='left')

In [6]:
ben_s1_state_cara.head()

,DESYNPUF_ID,BENE_BIRTH_DT,BENE_DEATH_DT,BENE_SEX_IDENT_CD,BENE_RACE_CD,BENE_ESRD_IND,SP_STATE_CODE,BENE_COUNTY_CD,BENE_HI_CVRAGE_TOT_MONS,BENE_SMI_CVRAGE_TOT_MONS,...,LINE_ICD9_DGNS_CD_4,LINE_ICD9_DGNS_CD_5,LINE_ICD9_DGNS_CD_6,LINE_ICD9_DGNS_CD_7,LINE_ICD9_DGNS_CD_8,LINE_ICD9_DGNS_CD_9,LINE_ICD9_DGNS_CD_10,LINE_ICD9_DGNS_CD_11,LINE_ICD9_DGNS_CD_12,LINE_ICD9_DGNS_CD_13
0,001D0E59C94130D3,19241201,NaN,2,1,0,31,290,12,12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,001D0E59C94130D3,19241201,NaN,2,1,0,31,290,12,12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,001D0E59C94130D3,19241201,NaN,2,1,0,31,290,12,12,...,4011,4011,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,001D0E59C94130D3,19241201,NaN,2,1,0,31,290,12,12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,001D0E59C94130D3,19241201,NaN,2,1,0,31,290,12,12,...,V0481,V0481,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# making changes to some data types
ben_s1_state_op['CLM_FROM_DT'] = ben_s1_state_op['CLM_FROM_DT'].apply(str)
ben_s1_state_op['CLM_FROM_YR'] = ben_s1_state_op['CLM_FROM_DT'].str.slice(0, 4)

ben_s1_state_cara['CLM_FROM_DT'] = ben_s1_state_cara['CLM_FROM_DT'].apply(str)
ben_s1_state_cara['CLM_FROM_YR'] = ben_s1_state_cara['CLM_FROM_DT'].str.slice(0, 4)


In [8]:
## Identification of ER Visits in Outpatient Claims Data file
#http://www.medicalbillingcptmodifiers.com/2013/01/emergency-department-cpt-codes-99281.html
ben_s1_state_op[
    (ben_s1_state_op.HCPCS_CD_1.isin(['G0380','G0381','G0382','G0383','G0384','99281','99282','99283','99284','99285'])) 
    & (ben_s1_state_op.CLM_FROM_YR == '2008')
].groupby(['HCPCS_CD_1','CLM_FROM_YR'])['CLM_ID'].count().reset_index(name='count').sort_values(['count'], ascending=False)

,HCPCS_CD_1,CLM_FROM_YR,count
1,99282,2008,33
2,99283,2008,32
0,99281,2008,19
3,99284,2008,5
4,99285,2008,4
5,G0380,2008,3
6,G0381,2008,3
7,G0382,2008,3


In [9]:
## Identification of ER Visits in Carrier Claims Data file
ben_s1_state_cara[
    (ben_s1_state_cara.HCPCS_CD_1.isin(['G0380','G0381','G0382','G0383','G0384','99281','99282','99283','99284','99285'])) 
    & (ben_s1_state_cara.CLM_FROM_YR == '2008')
].groupby(['HCPCS_CD_1','CLM_FROM_YR'])['CLM_ID'].count().reset_index(name='count').sort_values(['count'], ascending=False)

,HCPCS_CD_1,CLM_FROM_YR,count
4,99285,2008,260
3,99284,2008,160
2,99283,2008,132
1,99282,2008,23
0,99281,2008,2


In [10]:
s1_state_cara_er = ben_s1_state_cara[
    (ben_s1_state_cara.HCPCS_CD_1.isin(['G0380','G0381','G0382','G0383','G0384','99281','99282','99283','99284','99285']))]
s1_state_op_er = ben_s1_state_op[
    (ben_s1_state_op.HCPCS_CD_1.isin(['G0380','G0381','G0382','G0383','G0384','99281','99282','99283','99284','99285']))]

print(s1_state_cara_er['CLM_ID'].count()) ## 1,571 claims
print(s1_state_cara_er['CLM_ID'].nunique()) ## 1,571 claims

print(s1_state_op_er['CLM_ID'].count()) ## 268 claims
print(s1_state_op_er['CLM_ID'].nunique()) ## 268 claims


1571
1571
268
268


In [11]:
both_claimssets = pd.merge(s1_state_cara_er,s1_state_op_er,on='CLM_ID')

In [12]:
len(both_claimssets.index) ## there are no ER claims with the same claim ID in both datasets

0

In [85]:
#ben_s1_state_cara[ben_s1_state_cara['HCPCS_CD_1'].isin(['99281','99282','99283','99284','99285'])]['TAX_NUM_1'].value_counts()

#ben_s1_state_cara[ben_s1_state_cara['HCPCS_CD_1'].isin(['99281','99282','99283','99284','99285'])].groupby(['SP_STATE_CODE','TAX_NUM_1']).count()

ben_s1_state_cara[ben_s1_state_cara['HCPCS_CD_1'].isin(['99281','99282','99283','99284','99285'])]['TAX_NUM_1'].value_counts()

#.groupby(['SP_STATE_CODE','TAX_NUM_1']).count()


322988663.0    76
532092265.0    70
294540272.0    37
335709939.0    30
707841098.0    22
528517496.0    13
548420167.0    12
943326701.0    10
805904691.0     9
78712490.0      8
488493035.0     8
569180805.0     7
117749059.0     7
673314266.0     7
743332313.0     6
548970950.0     6
672331052.0     6
537716678.0     6
853981253.0     6
650282132.0     6
907765507.0     6
597692578.0     6
874054422.0     5
838615373.0     5
900667747.0     5
431847709.0     5
341190057.0     5
469131579.0     4
598117605.0     4
793111801.0     4
               ..
752008276.0     1
91903227.0      1
923967605.0     1
754177396.0     1
121535252.0     1
16144292.0      1
463089639.0     1
346706231.0     1
406474036.0     1
970233236.0     1
859082277.0     1
485114918.0     1
489308235.0     1
335165973.0     1
986999917.0     1
408564504.0     1
814904901.0     1
207951347.0     1
137166018.0     1
622086828.0     1
272245627.0     1
695613904.0     1
823405953.0     1
638851863.0     1
242025543.

In [100]:
ben_s1_cara[ben_s1_cara['TAX_NUM_1'].isin([322988663])].groupby(['SP_STATE_CODE','TAX_NUM_1'])['CLM_ID'].count().reset_index(name='count').sort_values(['count'], ascending=False)

#,532092265

,SP_STATE_CODE,TAX_NUM_1,count
52,34,532092265.0,5789
49,33,322988663.0,4855
72,49,532092265.0,4247
47,31,532092265.0,3517
19,11,532092265.0,3512
1,1,532092265.0,3169
46,31,322988663.0,3135
63,42,532092265.0,2595
28,18,532092265.0,2377
55,36,532092265.0,1754


In [101]:
ben_s1_cara[ben_s1_cara['TAX_NUM_1'].isin([322988663])].groupby(['SP_STATE_CODE','BENE_COUNTY_CD'])['CLM_ID'].count().reset_index(name='count').sort_values(['count'], ascending=False)

,SP_STATE_CODE,BENE_COUNTY_CD,count
86,33,420,969
57,31,100,775
90,33,590,699
84,33,331,698
104,33,800,604
74,33,20,566
64,31,270,354
65,31,290,344
92,33,610,325
67,31,310,266
